In [1]:
# Libs intialization
import sklearn as sk
import pandas as pd
import numpy as np
import math

from prettytable import PrettyTable
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df = pd.read_csv('Data/FilteredData.csv', sep= ';', index_col=False)
training_data, testing_data = train_test_split(df,random_state = 2000,)
Y_train=training_data['Class'].values
Y_test=testing_data['True class'].values
df.head(7)

,CleanedText,TokenizedText,Words quantity,Class,True class
0,day thor command pick copy sex american male h...,"['day', 'thor', 'command', 'pick', 'copy', 'se...",12,1,3
1,unfortunately due large number covid case clos...,"['unfortunately', 'due', 'large', 'number', 'c...",17,0,0
2,honourable prime minister shri narendra modi w...,"['honourable', 'prime', 'minister', 'shri', 'n...",17,1,1
3,week worked adding new deep ball shoulder catc...,"['week', 'worked', 'adding', 'new', 'deep', 'b...",14,3,0
4,area 51 game 2005 mass surveillance new world ...,"['area', '51', 'game', '2005', 'mass', 'survei...",11,3,3
5,one wave country politician brought going end ...,"['one', 'wave', 'country', 'politician', 'brou...",8,1,1
6,style truly described effortless,"['style', 'truly', 'described', 'effortless']",4,2,2


In [3]:
# TF-IDF Transformation
tf_idf=TfidfVectorizer(use_idf=True, max_df=0.95)
tf_idf.fit_transform(training_data['CleanedText'].values)
X_train=tf_idf.transform(training_data['CleanedText'].values)
X_test=tf_idf.transform(testing_data['CleanedText'].values)

In [4]:
# Calculation of accuracy
def acc_C(eval_items):
    correct=0
    for item in eval_items:
        true_pred=item[0]
        machine_pred=set(item[1])
        for cat in true_pred:
            if cat in machine_pred:
                correct+=1
                break
    accuracy=correct/float(len(eval_items))
    return accuracy

# rr_total = 0
# for item in eval_items:   

#     # add index to list only if machine predicted label exists in true labels
#     tp_pos_list = [(idx + 1) for idx, r in enumerate(item[1]) if r in item[0]]

#     rr_at_k = 0
#     if len(tp_pos_list) > 0:
#         # for RR we need position of first correct item
#         first_pos_list = tp_pos_list[0]
    
#         # rr = 1/rank
#         rr_at_k = 1 / float(first_pos_list)
#     rr_total = rr_total + rr_at_k
#     mrr = rr_total / 1/float(len(eval_items))
    

In [5]:
result  = []
for i in range (0, 5):
    if i == 0:
        # Logistic Regression
        model = LogisticRegression(verbose=1, solver='liblinear',random_state=0, C=5, penalty='l2', max_iter=1000)
    elif i == 1:
        # Logistic Regression
        model = KNeighborsClassifier (n_neighbors=3)    
    elif i == 2:
        # Logistic Regression
        model = KNeighborsClassifier (n_neighbors=3)  
    model=model.fit(X_train,Y_train)
    probs = model.predict_proba(X_test)
    best_n = np.argsort(probs, axis=1)[:,-3:]
    preds=[[model.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]
    preds=[ item[::-1] for item in preds]
    # Making a Comparative List of Truth and Predictions
    eval_items=[[[Y_test[idx]],pred] for idx,pred in enumerate(preds)]
    result.append([model, acc_C(eval_items)])

[LibLinear]iter  1 act 3.533e+02 pre 3.349e+02 delta 1.272e+00 f 1.279e+03 |g| 5.571e+02 CG   1
cg reaches trust region boundary
iter  2 act 5.625e+01 pre 5.556e+01 delta 3.446e+00 f 9.255e+02 |g| 7.053e+01 CG   2
cg reaches trust region boundary
iter  3 act 1.208e+02 pre 1.228e+02 delta 9.626e+00 f 8.693e+02 |g| 4.801e+01 CG   2
cg reaches trust region boundary
iter  4 act 2.066e+02 pre 2.014e+02 delta 1.319e+01 f 7.485e+02 |g| 7.088e+01 CG   2
iter  5 act 2.940e+01 pre 2.863e+01 delta 1.319e+01 f 5.419e+02 |g| 2.675e+01 CG   2
iter  6 act 4.730e-01 pre 4.714e-01 delta 1.319e+01 f 5.125e+02 |g| 6.166e+00 CG   2
iter  7 act 1.620e-02 pre 1.619e-02 delta 1.319e+01 f 5.120e+02 |g| 2.658e-01 CG   3
iter  8 act 5.968e-05 pre 5.968e-05 delta 1.319e+01 f 5.120e+02 |g| 1.560e-02 CG   3
iter  1 act 3.671e+02 pre 3.473e+02 delta 1.298e+00 f 1.279e+03 |g| 5.672e+02 CG   1
cg reaches trust region boundary
iter  2 act 6.005e+01 pre 5.924e+01 delta 3.444e+00 f 9.118e+02 |g| 7.387e+01 CG   2
cg reac

In [6]:
print(result)

[[LogisticRegression(C=5, max_iter=1000, random_state=0, solver='liblinear',
                   verbose=1), 0.8467741935483871], [KNeighborsClassifier(n_neighbors=3), 0.8145161290322581], [KNeighborsClassifier(n_neighbors=3), 0.8145161290322581], [KNeighborsClassifier(n_neighbors=3), 0.8145161290322581], [KNeighborsClassifier(n_neighbors=3), 0.8145161290322581]]
